### bilinear interpolation

In [27]:
from math import (ceil, floor)

In [33]:
def bilinear_interpolate(x, y, img):
    x2 = int(ceil(x))
    x1 = int(floor(x))
    y2 = int(ceil(y))
    y1 = int(floor(y))
    
    if x2 >= img.shape[0] or y2 >= img.shape[1]:
        return 0
    
    diff_x = x2 - x1
    diff_y = y2 - y1
    
    interpolate_x_y1 = -((x - x2) / (diff_x)) * img[x1, y1] + ((x - x1) / (diff_x)) * img[x2, y1]
    interpolate_x_y2 = -((x - x2) / (diff_x)) * img[x1, y2] + ((x - x1) / (diff_x)) * img[x2, y2]
    imterpolate_y = -((y - y2) / (diff_y)) * interpolate_x_y1 + ((y - y1) / (diff_y)) * interpolate_x_y2
    
    return imterpolate_y

In [35]:
def nearest_neighbor_interpolate(x, y, img):
    return img[int(x), int(y)]

### rotation

In [4]:
import numpy as np
import cv2

In [23]:
img = cv2.imread('khatiko.jpg')
grayed_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [36]:
def rotate(img, alpha):
    
    A_alpha = np.array([[np.cos(alpha), -np.sin(alpha)], [np.sin(alpha), np.cos(alpha)]])
    A_alpha_inv = np.linalg.inv(A_alpha)
    
    H, W = img.shape
    
    A_p = np.zeros_like(img)
    
    for i in range(H):
        for j in range(W):
            uniform_arr = [(2*i + 1) / (2*H), (2*j + 1) / (2*W)]
            x, y = np.dot(A_alpha_inv, uniform_arr)
            
            if 0 <= x < 1 and 0 <= y < 1:
                #A_p[i, j] = bilinear_interpolate(x*H, y*W, img)  #img[int(x * H), int(y * W)]
                A_p[i, j] = nearest_neighbor_interpolate(x*H, y*W, img)
                
    
    return A_p

In [14]:
def rotate_3_channels(img, alpha, b = [0, 0]):
    A_alpha = np.array([[np.cos(alpha), -np.sin(alpha)],
                     [np.sin(alpha), np.cos(alpha)]])
  
    A_alpha_inv = np.linalg.inv(A_alpha)
  
    H, W, C = img.shape
    A_p = np.zeros_like(img)
  
    for i in range(H):
        for j in range(W):
            for k in range(C):
                x, y = np.dot(A_alpha_inv, [(2 * i + 1) / (2 * H) - b[0], (2 * j + 1) / (2 * W) - b[1]])
                x, y = x + b[0], y + b[1]
                if 0 <= x < 1 and 0 <= y < 1:
                    print('x = ', 'y = ')
                    A_p[i, j, k] = img[int(x*H), int(y*W), k]
  
    return A_p

In [37]:
cv2.imwrite('khatiko_rotated.jpg', rotate(grayed_image, np.deg2rad(40)))

True

In [40]:
def resize_img(img, shape):
    H_n, W_n = shape
    H, W = img.shape
    
    A_p = np.zeros(shape)
    
    for i in range(H_n):
        for j in range(W_n):
            x, y = (2*i + 1) / (2*H_n), (2*j + 1) / (2*W_n)
            A_p[i, j] = bilinear_interpolate(x)
            #nearest_neighbor_interpolate(x*H, y*W, img)
    
    return A_p

In [41]:
cv2.imwrite('khatiko_resized.jpg', resize_img(grayed_image, (500, 500)))

True

### erosion

In [1]:
import numpy as np
from scipy import ndimage

In [2]:
input_array = np.ones((7,7), dtype=np.int)
input_array[3,0] = 0
input_array[1,3:5] = 0

In [3]:
input_array

array([[1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1]])

In [4]:
structure = np.ones((3,3), dtype=np.int)

In [5]:
ndimage.binary_erosion(input_array, structure)

array([[False, False, False, False, False, False, False],
       [False,  True, False, False, False, False, False],
       [False, False, False, False, False, False, False],
       [False, False,  True,  True,  True,  True, False],
       [False, False,  True,  True,  True,  True, False],
       [False,  True,  True,  True,  True,  True, False],
       [False, False, False, False, False, False, False]])

In [6]:
pad_array = np.zeros((9,9), dtype=np.int)
pad_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [7]:
pad_array[1:-1,1:-1] = input_array
pad_array

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [16]:
def np_binary_erosion(input_array, structure=np.ones((3,3)).astype(np.bool)):
    '''NumPy binary erosion function
     
    No error checking on input array (type)
    No error checking on structure element (# of dimensions, shape, type, etc.)
     
    Args:
    input_array: Binary NumPy array to be eroded. Non-zero (True) elements
        form the subset to be eroded
    structure: Structuring element used for the erosion. Non-zero elements
        are considered True. If no structuring element is provided, an
        element is generated with a square connectivity equal to two 
        (square, not cross).
    Returns:
        binary_erosion: Erosion of the input by the stucturing element
    '''
    rows, cols = input_array.shape
     
    ## Pad output array (binary_erosion) with extra cells around the edge
    ## so that structuring element will fit without wrapping.
    ## A 3x3 structure, will need 1 additional cell around the edge
    ## A 5x5 structure, will need 2 additional cells around the edge
    
    input_shape, structure_shape = input_array.shape, structure.shape
    
    pad_shape = (
        input_shape[0] + structure_shape[0] - 1, 
        input_shape[1] + structure.shape[1] - 1)
    input_pad_array = np.zeros(pad_shape).astype(np.bool)
    input_pad_array[1:rows+1,1:cols+1] = input_array
    binary_erosion = np.zeros(pad_shape).astype(np.bool)
     
    ## Cast structure element to boolean
    struc_mask = structure.astype(np.bool)

    ## Iterate over each cell
    for row in range(rows):
        for col in range(cols):
            ## The value of the output pixel is the minimum value of all the
            ## pixels in the input pixel's neighborhood.
            print('input_pad_array', input_pad_array[row:row+3, col:col+3])
            print('struc_mask', struc_mask)
            print('res', input_pad_array[row:row+3, col:col+3][struc_mask])
            binary_erosion[row+1,col+1] = np.min(
                input_pad_array[row:row+3, col:col+3][struc_mask])
    return binary_erosion[1:rows+1,1:cols+1]

In [17]:
np_binary_erosion(input_array, structure).astype(np.int)

input_pad_array [[False False False]
 [False  True  True]
 [False  True  True]]
struc_mask [[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
res [False False False False  True  True False  True  True]
input_pad_array [[False False False]
 [ True  True  True]
 [ True  True  True]]
struc_mask [[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
res [False False False  True  True  True  True  True  True]
input_pad_array [[False False False]
 [ True  True  True]
 [ True  True False]]
struc_mask [[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
res [False False False  True  True  True  True  True False]
input_pad_array [[False False False]
 [ True  True  True]
 [ True False False]]
struc_mask [[ True  True  True]
 [ True  True  True]
 [ True  True  True]]
res [False False False  True  True  True  True False False]
input_pad_array [[False False False]
 [ True  True  True]
 [False False  True]]
struc_mask [[ True  True  True]
 [ True  True  True]
 [ True

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [ ]:
def dilation(img):
    
    ### Dilation of image: max{a, b} = ( |a-b| + (a+b)) /2
    
    # deep copy of argument, because of numpy shallow copying property
    img = copy.deepcopy(img)
    
    # pad image by 0-values
    img_pad = np.hstack((img, np.zeros((img.shape[0], 1))))
    
    kernel_1 = np.array([[1,  1]])
    kernel_2 = np.array([[1, -1]])
    
    s = convolve2d(img_pad, kernel_1, boundary='symm', mode='same')
    b = convolve2d(img_pad, kernel_2, boundary='symm', mode='same')

            
    return (np.abs(b) + s) / 2

In [ ]:
def erosion(img):
    
    #### Erosion of image: min{a, b} = ( - |a-b| + (a+b) ) /2
    
    # deep copy of argument, because of numpy shallow copying property
    img = copy.deepcopy(img)
    
    # pad image by 0-values
    img_pad = np.hstack((img, np.zeros((img.shape[0], 1))))
    
    kernel_1 = np.array([[1,  1]])
    kernel_2 = np.array([[1, -1]])
    
    s = convolve2d(img_pad, kernel_1, boundary='symm', mode='same')
    b = convolve2d(img_pad, kernel_2, boundary='symm', mode='same')
            
    return (- np.abs(b) + s) / 2